<h2>Taller 5 - Cinemática diferencial para tareas de soldadura</h2>
<br>Autor: Claudio Morales D.
<br>Centro de Automatización y Robótica Inacap
<br>Santiago, Chile, enero 2023


Desde CoppeliaSim, abrir la escena 'scara_welding.ttt'
<br> La escena contiene un script que agrega un elemento 'DrawingObject' para dibujar un trazo del movimiento realizado por la punta del robot. Para más información sobre este elemento, consulte <a>https://www.coppeliarobotics.com/helpFiles/en/regularApi/simAddDrawingObject.htm</a>
<br>Los conceptos de cinemática directa y inversa pueden consultarse en este enlace: https://youtu.be/G2A2Zm7BfXY
<br>En una actividad previa se muestra la obtención de la cinemática directa del robot: https://youtu.be/ZiRuWQo7EzA 

In [ ]:
# importamos las librerías necesarias
import sympy
import time

###  Conectamos con CoppeliaSim y obtenemos los manejadores
Utilizamos las funciones del API Remoto de CoppeliaSim. Para más detalles refiérase a la documentación de la librería:
<br> https://www.coppeliarobotics.com/helpFiles/en/apiFunctions.htm

In [ ]:
# Cargamos la librería, creamos un cliente y obtenemos acceso a sim
import zmqRemoteApi
client = zmqRemoteApi.RemoteAPIClient()
sim = client.getObject('sim')

# Obtenemos los manejadores para las articulaciones y el actuador final
j1 = sim.getObject('/base/joint1')
j2 = sim.getObject('/base/joint2')
j3 = sim.getObject('/base/joint3')
tip = sim.getObject('/base/tip')

print(j1, j2, j2, tip)

# ponemos el robot en posición inicial
sim.setJointTargetPosition(j1, 0)
sim.setJointTargetPosition(j2, 0.2)
sim.setJointTargetPosition(j3, 0)

<h3> 1. Obtenemos la cinemática directa</h3>
Conocidas las posiciones de las articulaciones, podemos conocer la posición final del actuador.

In [ ]:
# Comenzaremos el trabajo desde la descripción de Denavit-Hartenberg para el robot
#  link |     theta     |      d      |      a      |    alpha
# --------------------------------------------------------------
#   1           q1           0.13           0            0
#   2            0             q2         0.2            0
#   3           q3          -0.052        0.2           180
#  -------------------------------------------------------------
#   4           0            0.068       0.013          180

In [ ]:
# Definimos una función para construir las matrices de transformación
# en forma simbóĺica a partir de los parámetros D-H

def MatrixfromDH(theta, d, a, alpha):
    # theta y alpha en radianes
    # d y a en metros
    Rz = sympy.Matrix([[sympy.cos(theta), -sympy.sin(theta), 0, 0],
                   [sympy.sin(theta), sympy.cos(theta), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    tz = sympy.Matrix([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, d],
                   [0, 0, 0, 1]])
    ta = sympy.Matrix([[1, 0, 0, a],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    Rx = sympy.Matrix([[1, 0, 0, 0],
                   [0, sympy.cos(alpha), -sympy.sin(alpha), 0],
                   [0, sympy.sin(alpha), sympy.cos(alpha), 0],
                   [0, 0, 0, 1]])
    return Rz*tz*ta*Rx

# y definimos una función para obtener la matriz de transformación desde una pose

def matrixFromPose(pose):
    # pose = [x, y, z, alpha, beta, gamma]
    # x, y, z en metros
    # alpha, beta, gamma en radianes
    x, y, z = pose[0], pose[1], pose[2]
    alpha, beta, gamma = pose[3], pose[4], pose[5]
    Ra = sympy.Matrix([[1, 0, 0, 0],
                   [0, sympy.cos(alpha), -sympy.sin(alpha), 0],
                   [0, sympy.sin(alpha), sympy.cos(alpha), 0],
                   [0, 0, 0, 1]])
    Rb = sympy.Matrix([[sympy.cos(beta), 0, sympy.sin(beta), 0],
                   [0, 1, 0, 0],
                   [-sympy.sin(beta), 0, sympy.cos(beta), 0],
                   [0, 0, 0, 1]])
    Rc = sympy.Matrix([[sympy.cos(gamma), -sympy.sin(gamma), 0, 0],
                   [sympy.sin(gamma), sympy.cos(gamma), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    T = Ra*Rb*Rc
    T[0,3] = x
    T[1,3] = y
    T[2,3] = z
    return T

In [ ]:
# Obtenemos la matriz de transformación de la base al extremo del robot
# desde los parámetros D-H
q1, q2, q3 = sympy.symbols(['q1', 'q2', 'q3'])
j1_T_j2 = MatrixfromDH(q1, 0.13, 0, 0)
j2_T_j3 = MatrixfromDH(0, q2, 0.2, 0)
j3_T_conn = MatrixfromDH(q3, -0.052, 0.2, sympy.pi)
conn_T_tip = MatrixfromDH(0, 0.068, 0.013, sympy.pi)
j1_T_tip = sympy.simplify(j1_T_j2 * j2_T_j3 * j3_T_conn * conn_T_tip)
j1_T_tip

In [ ]:
# Y simplificamos la matriz para ocuparnos únicamente de las posiciones
T = matrixFromPose([j1_T_tip[3], j1_T_tip[7], j1_T_tip[11], 0, 0, 0])
T

<h3> 2. Posicionamos el robot mediante cinemática inversa</h3>

In [ ]:
# Obtenemos los manejadores para todos los elementos requeridos
j1 = sim.getObject('/base/joint1')
j2 = sim.getObject('/base/joint2')
j3 = sim.getObject('/base/joint3')
tip = sim.getObject('/base/tip')
target = sim.getObject('/caja')

print(j1, j2, j2, tip, target)

# ponemos el robot en posición inicial
sim.setJointTargetPosition(j1, 0)
sim.setJointTargetPosition(j2, 0.2)
sim.setJointTargetPosition(j3, 0)

In [ ]:
# definimos las coordenadas de destino
pos = sim.getObjectPosition(target, sim.handle_world)
a = pos[0] + 0.025
b = pos[1] + 0.025
c = pos[2] + 0.052
pose_destino = [a, b, c, 0, 0, 0]  # no consideramos la orientación
pose_destino

In [ ]:
#la matriz de transformación para las coordenadas de destino será
D = matrixFromPose(pose_destino)
D

In [ ]:
# ahora resolvemos la ecuación utilizando nsolve()
q = sympy.nsolve(T-D, (q1, q2, q3), (0.1, 0.1, 0.1), prec=6)
q

In [ ]:
# y ponemos el robot en posición
sim.setJointTargetPosition(j1, float(q[0]))
sim.setJointTargetPosition(j2, float(q[1]))
sim.setJointTargetPosition(j3, float(q[2]))

<h3> 3. Ahora calculamos la cinemática diferencial en torno al punto de interés...</h3>

In [ ]:
# Definimos la función f(q)


In [ ]:
# Calculamos el jacobiano


In [ ]:
# Evaluamos el jacobiano para las posiciones actuales de q


In [ ]:
# calculamos la inversa


In [ ]:
# definimos el vector de desplazamiento dx


In [ ]:
# calculamos las variaciones de las articulaciones


In [ ]:
# ajustamos las posiciones de las articulaciones


In [ ]:
# y ponemos el robot en posición


<h3> 4. Y ponemos todo junto en una rutina</h3>